# Further Studies

### Laser Identification

Laser identified, 100Hz. Hard to tell on such a long plot that the data does not overlap locally, but it works. Looks like there's reasonable amounts of off-laser data - maybe it's nuclear!

What about the events near zero?

In [ ]:
%%capture
%%
"""The data near zero time is interesting.

The vertical discretization might be some form of time binning.
The fact that it's at near zero absolute time is strange. Is there some sort of large noise upon startup?
"""

laser_hz = 100
df["ig_laser"] = np.abs(
    ((df.realtime - df.realtime.min()).dt.total_seconds() % (1 / laser_hz))
    - ((df.realtime - df.realtime.min()).dt.total_seconds() % (1 / laser_hz))
    .rolling(1000)
    .median()
) < 0.05 * np.mean(
    (df.realtime - df.realtime.min()).dt.total_seconds() % (1 / laser_hz)
)


fig, ax = plt.subplots(figsize=(10, 6), dpi=200, constrained_layout=True)
ax.scatter(
    (df.realtime - df.realtime.min()).dt.total_seconds(),
    (df.realtime - df.realtime.min()).dt.total_seconds() % (1 / laser_hz),
    s=0.1,
    lw=0,
    alpha=0.5,
)
data = df[df.ig_laser]
ax.scatter(
    (data.realtime - df.realtime.min()).dt.total_seconds(),
    (data.realtime - df.realtime.min()).dt.total_seconds() % (1 / laser_hz),
    s=0.1,
    lw=0,
    alpha=0.5,
)
ax.set(
    xlim=(-5, 10),
    xlabel="Time since start (s)",
    ylabel=f"Time modulo 1/{laser_hz} s",
    title=f"Laser Identification for {DATE}",
)
fig.savefig(os.path.join(figdir, f"laserIdentification_{laser_hz}Hz.png"))

In [ ]:
%%capture
%%
"""The data near zero is nothing interesting I guess.
Lots of it are the 'bad' events tagged by setting their height to zero. Be sure to filter those out,
otherwise you get a large line at zero.
"""

fig, ax = plt.subplots(figsize=(10, 4), dpi=200, constrained_layout=True)
data = df[(df.realtime - df.realtime.min()).dt.total_seconds() < 5.5]
h, b = np.histogram(data.height_mV, bins=1000)
ax.step(b[:-1], h, where="mid")
data = df[(df.realtime - df.realtime.min()).dt.total_seconds() < 5]
h, b = np.histogram(data.height_mV, bins=1000)
ax.step(b[:-1], h, where="mid")

## Nuclear Data

## Using Laser Data!

In [ ]:
"""Per Channel Nuclear Data

~ig_laser

"""

channels = df.channel.unique()
fig, axes = plt.subplots(
    len(channels),
    1,
    figsize=(8, 3.2 * len(channels)),
    dpi=DPI_VIS,
    constrained_layout=True,
)
for channel, ax in list(zip(channels, np.ravel(axes))):
    range = (0, 50)
    bins = 1000
    if channel == 27:
        bins = 350
        range = (0, 25)
    data = df[(df.channel == channel) & (~df.ig_laser)]
    h, b = np.histogram(data.height_mV, bins=bins, range=range)
    ax.step(b[:-1], h, where="post", label="All Laser-Anticoincident Data", zorder=-21)
    for mxmult in [2, 4, 6, 8, 10, 15, 20]:
        data = df[(df.channel == channel) & (~df.ig_laser) & (df.multiplicity < mxmult)]
        h, b = np.histogram(data.height_mV, bins=bins, range=range)
        ax.step(
            b[:-1], h, where="post", label=f"Multiplicity < {mxmult}", zorder=-mxmult
        )
    ax.set(
        title=f"Channel {channel}",
        xlabel="Height [mV]",
        ylabel="Counts",
        yscale="log",
    )
    ax.legend()
display(fig)
fig.savefig(os.path.join(figdir, "per_channel-nuclear_data-laser_anticoincidence.png"))
print(
    f"Saved {os.path.join(figdir, 'per_channel-nuclear_data-laser_anticoincidence.png')}"
)
plt.close(fig)


## Original Method w Mulitplicity

In [ ]:
%%capture
%%
"""Per Channel Nuclear Data

Multiplicity 1
"""

num_channels = df.channel.unique().size
fig, axes = plt.subplots(
    num_channels, 1, figsize=(5, 3 * num_channels), dpi=DPI_VIS, constrained_layout=True
)
for channel, ax in list(zip(df.channel.unique(), np.ravel(axes))):
    data = df[(df.channel == channel) & (df.multiplicity == 1)]
    range = (0, 50)
    bins = 1000
    if channel == 27:
        bins = 350
        range = (0, 25)
    h, b = np.histogram(data.height_mV, bins=bins, range=range)
    ax.step(b[:-1], h, where="post", label="All Data")
    ax.set(
        title=f"Channel {channel}",
        xlabel="Height [mV]",
        ylabel="Counts",
        yscale="log",
    )
display(fig)
fig.savefig(os.path.join(figdir, "per_channel-nuclear_data-m1.png"))
plt.close(fig)

In [ ]:
%%capture
%%
"""Per Channel Multiplicity Ranges

Multiplicities
- 2 to nchannels/4
- nchannels/4 to nchannels/2
- nchannels/2 to 3nchannels/4
- 3nchannels/4 to nchannels
"""

num_channels = df.channel.unique().size
fig, axes = plt.subplots(
    num_channels,
    1,
    figsize=(10, 6 * num_channels),
    dpi=DPI_VIS,
    constrained_layout=True,
)
for channel, ax in list(zip(df.channel.unique(), np.ravel(axes))):
    data = df[(df.channel == channel) & (df.multiplicity == 1)]
    range = (0, 50)
    bins = 1000
    if channel == 27:
        bins = 350
        range = (0, 25)
    h, b = np.histogram(data.height_mV, bins=bins, range=range)
    ax.step(b[:-1], h, where="post", label="Nuclear Data")
    ax.set(
        title=f"Channel {channel}",
        xlabel="Height [mV]",
        ylabel="Counts",
        yscale="log",
    )
    for multiplicity in np.linspace(2, num_channels, 4):
        data = df[
            (df.channel == channel)
            & (df.multiplicity >= multiplicity)
            & (df.multiplicity < multiplicity + num_channels / 4)
        ]
        h, b = np.histogram(data.height_mV, bins=bins, range=range)
        ax.step(
            b[:-1],
            h,
            where="post",
            label=f"Mult. {multiplicity:.01f} - {multiplicity + num_channels / 4:.01f}",
        )
    ax.legend()
display(fig)
fig.savefig(os.path.join(figdir, "per_channel-nuclear_data-mX.png"))
plt.close(fig)

## Redo Multiplicity and Save

In [ ]:
%%capture
%%
"""Recalculate Multiplicity by Threshold

Plot resulting sums of data
"""

fig, ax = plt.subplots(figsize=(10, 6), dpi=DPI_VIS, constrained_layout=True)
for slow_threshold in np.linspace(0, 3, 5):
    data = df[df.channel != 27]
    multiplicity = data.groupby("eventID").height_mV.apply(
        lambda x: (x > slow_threshold).sum()
    )
    data = data[data.eventID.isin(multiplicity[multiplicity == 1].index)]
    h, b = np.histogram(data.height_mV, bins=1000, range=(0, 35))
    ax.step(
        b[:-1],
        h,
        where="post",
        lw=0.5,
        alpha=0.7,
        label=f"slow_amp > {slow_threshold:.2f}",
    )
ax.set(
    title="Multiplicity 1 Data",
    xlabel="Height [mV]",
    ylabel="Counts",
    yscale="log",
)
ax.legend()
display(fig)
fig.savefig(os.path.join(figdir, "sum-nuclear_data-m1_threshold.png"))
plt.close(fig)

# Debug